In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, train_test_split, randomized_search_cv
from sklearn.metrics import mean_squared_error, mean_absolute_error

df =  pd.read_csv(r'C:\Users\kian3\Consulting\ORTEC\data\Basic_random= 1000,100.csv') # Adjust the path once data has been generated with all policies
df =  df.drop_duplicates(subset=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'cost'])

X = df[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14']]
y = df['cost']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

param_grid = {
    'num_leaves': [15, 31, 63],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [-1, 5, 10],  # -1 betekent geen limiet
    'min_child_samples': [5, 20, 50]  # Vergelijkbaar met min_samples_leaf
}

lgb_reg = lgb.LGBMRegressor(verbose=-1, random_state=42)

grid_search = GridSearchCV(
    estimator=lgb_reg,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

best_lgb = grid_search.best_estimator_
y_pred_tuned = best_lgb.predict(X_test)
print(f"Beste parameters: {grid_search.best_params_}")

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Squared Error: ", mse)
print("Mean Absolute Error: ", mae)


